In [1]:
# notebooks/data_exploration.ipynb

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the feature data
feature_df = pd.read_csv('../data/raw/secom.data', sep=' ', header=None)
feature_df = feature_df.reset_index(drop=True) # Reset index of feature_df

# Load the labels
# Load the labels CORRECTLY - target is the first column, timestamp is the second
labels_df = pd.read_csv('../data/raw/secom_labels.data', sep=' ', header=None, names=['target', 'timestamp'])
print("First 5 rows of labels_df (CORRECTED):")
print(labels_df.head())
print("\nData type of 'target' in labels_df (CORRECTED):")
print(labels_df['target'].dtype)

# Replace target values
labels_df['target'] = labels_df['target'].replace(-1, 0)
print("\nValue counts in labels_df after replacement (CORRECTED):")
print(labels_df['target'].value_counts())
print("\nData type of 'target' in labels_df after replacement (CORRECTED):")
print(labels_df['target'].dtype)

# We don't need the timestamp for the model, so we can drop it
labels_df = labels_df.drop('timestamp', axis=1)

labels_df = labels_df.reset_index(drop=True) # Ensure index of labels_df is also reset

# Explicitly assign the 'target' column to feature_df
df = feature_df.copy() # Start with a copy of feature_df
df['target'] = labels_df['target']

# --- Initial Data Inspection ---
print("\nShape of the combined DataFrame:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nTarget variable distribution:")
print(df['target'].value_counts())
print("\nNumber of missing values per column (top 10):")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# --- Identify Data Types ---
print("\nData types of all columns:")
print(df.dtypes)

# --- Handle Non-Numeric Column (Likely the Timestamp) ---
# Identify the column with object dtype (excluding 'target')
object_cols = df.select_dtypes(include=['object']).columns
object_feature_cols = [col for col in object_cols if col != 'target']

if object_feature_cols:
    print(f"\nNon-numeric feature columns found: {object_feature_cols}")
    # For the hackathon, let's simply remove these non-numeric feature columns
    df = df.drop(columns=object_feature_cols)
    print("\nDataFrame shape after removing non-numeric feature columns:", df.shape)
else:
    print("\nNo non-numeric feature columns found.")

# --- Handling Missing Values ---
# Remove completely NaN columns
df = df.drop(columns=[589, 590], errors='ignore')

# Identify columns with a high percentage of missing values (e.g., > 50%)
missing_percentage = (df.isnull().sum() / len(df)) * 100
high_missing_cols = missing_percentage[missing_percentage > 50].index
print("\nColumns with > 50% missing values:", list(high_missing_cols))

# Remove these high-missing-value columns
df = df.drop(columns=high_missing_cols, errors='ignore')
print("\nShape of DataFrame after removing high missing value columns:", df.shape)
print("\nNumber of missing values per column after removal (top 5):")
print(df.isnull().sum().sort_values(ascending=False).head())

# Impute remaining missing values using the median (only for numeric columns)
numeric_cols = df.select_dtypes(include=np.number).columns
for col in numeric_cols:
    if col != 'target':
        df[col] = df[col].fillna(df[col].median())

print("\nNumber of missing values after imputation:", df.isnull().sum().sum())

# --- Final check of 'target' column before saving ---
print("\nFinal check of 'target' column before saving:")
print(df['target'].head())
print("\nValue counts of 'target' before saving:")
print(df['target'].value_counts())
print("\nData type of 'target' before saving:")
print(df['target'].dtype)
print("\nUnique values in 'target' before saving:")
print(df['target'].unique())

# --- Save the preprocessed data ---
output_file = '../data/processed/secom_preprocessed.csv'
df.to_csv(output_file, index=False)
print(f"\nPreprocessed DataFrame saved to: {output_file}")

print("\nData loading, cleaning, and saving complete.")

First 5 rows of labels_df (CORRECTED):
   target            timestamp
0      -1  19/07/2008 11:55:00
1      -1  19/07/2008 12:32:00
2       1  19/07/2008 13:17:00
3      -1  19/07/2008 14:43:00
4      -1  19/07/2008 15:22:00

Data type of 'target' in labels_df (CORRECTED):
int64

Value counts in labels_df after replacement (CORRECTED):
0    1463
1     104
Name: target, dtype: int64

Data type of 'target' in labels_df after replacement (CORRECTED):
int64

Shape of the combined DataFrame: (1567, 591)

First 5 rows:
         0        1          2          3       4      5         6       7  \
0  3030.93  2564.00  2187.7333  1411.1265  1.3602  100.0   97.6133  0.1242   
1  3095.78  2465.14  2230.4222  1463.6606  0.8294  100.0  102.3433  0.1247   
2  2932.61  2559.94  2186.4111  1698.0172  1.5102  100.0   95.4878  0.1241   
3  2988.72  2479.90  2199.0333   909.7926  1.3204  100.0  104.2367  0.1217   
4  3032.24  2502.87  2233.3667  1326.5200  1.5334  100.0  100.3967  0.1235   

        8   